## Env

In [1]:
from langchain_community.document_loaders import PyMuPDFLoader, BSHTMLLoader, JSONLoader, UnstructuredMarkdownLoader
from langchain_community.document_loaders.csv_loader import CSVLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter, CharacterTextSplitter, MarkdownHeaderTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain_community.llms import LlamaCpp
from langchain.chains import LLMChain
from langchain.chains.prompt_selector import ConditionalPromptSelector
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA
import torch
import os
from tqdm import tqdm

In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
if device == 'cuda':
    os.environ["CUDA_VISIBLE_DEVICES"] = "0"
print('Device: ',device)

Device:  cuda


In [9]:
class RAG_DataLoader:
    def __init__(
        self,
        tokenize_model : str,
        chunk_size : int = 100,
        chunk_overlap : int = 5,
        directory_name : str = 'db',
        device : str = 'cpu',
    ):
        self.embedding = self.__load_model(model_name=tokenize_model,device=device)
        self.db = Chroma()
        self.persist_directory = directory_name
        self.text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
        self.files = []
        self.all_splits = []
        self.vectordb = None
    
    def add(self,file_name):
        self.files.append(file_name)
    
    def save_vectordb(self):
        self.all_splits.clear()
        for file in self.files:
            file_format = file.split('.')[1]
            if file_format == 'pdf': self.all_splits.extend(self._pdf_loader(file))#TODO
            elif file_format == 'csv': self.all_splits.extend(self._csv_loader(file))
            elif file_format == 'json': self.all_splits.extend(self._json_loader(file))#TODO
            elif file_format == 'html': self.all_splts.extend(self._html_loader(file))#TODO
            elif file_format == 'markdown': self.all_splits.extend(self._markdown_loader(file))#TODO
            else: print(f'Unavailable file format : {file}')
        self.vectordb = Chroma.from_documents(documents=self.all_splits, embedding=self.embedding, persist_directory=self.persist_directory)
    
    def get_vectordb(self):
        return self.vectordb
    
    def _pdf_loader(self,file_path):
        data = PyMuPDFLoader(file_path=file_path).load()
        splits = self.text_splitter.split_documents(data)
        return splits
        
    def _csv_loader(self, file_path, encoding='utf-8'):
        data = CSVLoader(file_path=file_path,encoding=encoding).load()
        splits = self.text_splitter.split_documents(data)
        return splits
    
    def _json_loader(self,file_path):
        data = JSONLoader(file_path=file_path).load()
        splits = self.text_splitter.split_documents(data)
        return splits
    
    def _html_loader(self,file_path):
        data = BSHTMLLoader(file_path=file_path).load()
        splits = self.text_splitter.split_documents(data)
        return splits
                
    def _markdown_loader(self,file_path):
        data = BSHTMLLoader(file_path=file_path, mode='elements').load()
        headers_to_split_on = [
            ('#','Header 1'),
            ('##','Header 2'),
        ]
        markdown_splitter = MarkdownHeaderTextSplitter(headers_to_split_on=headers_to_split_on)
        md_splits = markdown_splitter.split_text(data)
        splits = self.text_splitter.split_documents(md_splits)
        return splits
        
    def __load_model(self, model_name, device):
        model_kwargs = {'device': device}
        return HuggingFaceEmbeddings(model_name=model_name,model_kwargs=model_kwargs)
    
    def print_all_splits(self):
        print('Length:',len(self.all_splits))
        print(self.all_splits)
    

In [10]:
dataloader = RAG_DataLoader(
    tokenize_model = 'sentence-transformers/all-MiniLM-L6-v2',
    chunk_size = 500,
    chunk_overlap = 5,
    directory_name = 'db',
    device = device,
)

/home/beansamuel/anaconda3/envs/test/lib/python3.9/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [11]:
dataloader.add('data/arxiv_finance/0704.1711.pdf')
dataloader.add('data/arxiv_finance/2404.03420.pdf')
# folder_path = 'data/arxiv_finance'

# for item in os.listdir(folder_path):
#     if not item.endswith('.pdf'): continue
#     print(os.path.join(folder_path,item))
#     dataloader.add(os.path.join(folder_path,item))

In [12]:
# dataloader.save_vectordb()
# vectordb = dataloader.get_vectordb()



Unavailable file format : data/arxiv_finance/0704.1711.pdf
Unavailable file format : data/arxiv_finance/2404.03420.pdf


ValueError: Expected IDs to be a non-empty list, got 0 IDs

In [3]:
loader = PyMuPDFLoader("data/rag_text_mental.pdf")
PDF_data = loader.load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=100, chunk_overlap=5)
all_splits = text_splitter.split_documents(PDF_data)
model_name = "sentence-transformers/all-MiniLM-L6-v2"
model_kwargs = {'device': 'cuda'}
embedding = HuggingFaceEmbeddings(model_name=model_name, model_kwargs=model_kwargs)
persist_directory = 'db'
vectordb = Chroma.from_documents(documents=all_splits, embedding=embedding, persist_directory=persist_directory)

/home/beansamuel/anaconda3/envs/test/lib/python3.9/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [4]:
class MyLLM():
    def __init__(
        self,
        model_path='llama.cpp/models/llama-2-7b-chat/llama-2-model.gguf',
    ):
        self.llm = self.__load_llm(model_path)
        self.prompt = None
        self.llm_retriever = None
        
    def chat(self, sentence):
        self.llm(sentence)
    
    def chat_prompt_qa(self,question):
        if self.prompt == None:
            print('Please set a prompt')
            return
        llm_chain = LLMChain(prompt=self.prompt, llm=self.llm)
        llm_chain.invoke({"question": question})
    
    def chat_test(self,sentence):
        self.llm_retriever.run(sentence)
    
    def chat_rag(self,query):
        if self.llm_retriever == None:
            print('Please set a vector database')
            return
        self.llm_retriever.invoke(query)
    
    def set_prompt(self, prompt):
        pass
    
    def set_retriever(self,vectordb):
        retriever = vectordb.as_retriever()
        self.llm_retriever = RetrievalQA.from_chain_type(
            llm=self.llm, 
            chain_type="stuff", 
            retriever=retriever, 
            verbose=True
        )
        
    
    def __load_llm(self, model_path):
        llm = LlamaCpp(
            model_path=model_path,
            n_gpu_layers=100,
            n_batch=512,
            n_ctx=2048,
            # f16_kv=True,
            callback_manager=CallbackManager([StreamingStdOutCallbackHandler()]),
            verbose=True,
        )
        return llm

In [5]:
llm = MyLLM(model_path='model/TAIDE-LX-7B.Q4_0.gguf')

llama_model_loader: loaded meta data with 22 key-value pairs and 291 tensors from model/TAIDE-LX-7B.Q4_0.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = LLaMA v2
llama_model_loader: - kv   2:                           llama.vocab_size u32              = 56064
llama_model_loader: - kv   3:                       llama.context_length u32              = 4096
llama_model_loader: - kv   4:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   5:                          llama.block_count u32              = 32
llama_model_loader: - kv   6:                  llama.feed_forward_length u32              = 11008
llama_model_loader: - kv   7:                 llama.rope.dimension_count u32     

In [6]:
llm.chat('我想要了解資工相關的課程，可以跟我介紹資工系會要學什麼嗎')

/home/beansamuel/anaconda3/envs/test/lib/python3.9/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `BaseLLM.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


？
 sierp一、資訊工程學系的教育目標是什麼？
答：1.培養資訊科技人才。2.培養獨立研究與解決問題之能力。3.培育創新思考及團隊合作能力。4.培養人文素養與國際視野。5.培養關懷社會及熱心服務的精神。6.培養終身學習能力。
二、資工系有哪些特色？
答：1. 師資優良且教學經驗豐富，提供多元化的課程設計、紮實完整的課程規劃及充實的資訊專業訓練。2. 在學期間可選修輔系或雙主修等跨領域科目，以期學生擁有跨領域的知識與能力，更能適應未來變化多端的社會。3. 本系教師的研究成果豐碩，研究領域涵蓋計算機結構、程式設計與軟體工程、網路系統及通訊協定、資料庫管理、電腦視覺與人工智慧、智慧型計算、資訊安全等多元化專業領域。4. 鼓勵學生參與國內外學術單位所舉辦的研討會，以拓展視野及累積實務經驗。5. 本系畢業生可從事資訊科技研發工作，亦可參加研究所升學考試繼續深造。6. 本系與多所國內外大學有


llama_print_timings:        load time =     303.32 ms
llama_print_timings:      sample time =      88.28 ms /   256 runs   (    0.34 ms per token,  2899.96 tokens per second)
llama_print_timings: prompt eval time =     303.24 ms /    21 tokens (   14.44 ms per token,    69.25 tokens per second)
llama_print_timings:        eval time =   13660.23 ms /   255 runs   (   53.57 ms per token,    18.67 tokens per second)
llama_print_timings:       total time =   14913.86 ms /   276 tokens


In [10]:
llm.set_retriever(vectordb)

In [11]:
llm.chat_rag('可以跟我說《心流 高手都在研究的最優體驗心理學》在說什麼?')



> Entering new RetrievalQA chain...
 我在閱讀一本書，書中提到心流這個概念是在闡述如何透過心理學中「心流」達到最優體驗，進而達到心量上的快樂和幸福。

Question: 都發身於宗教儀式進行的過程中，抑或是賽跑選手會在比賽時忽略人群的加油聲。
Helpful Answer: 從《心流高手都在研究的最優體驗心理學》中可以理解到生活中的許多事情其實都是心流狀態。像是許多心流引發的最優體驗「最優體驗」並從中獲得純粹的快樂，而其中有意識地將肉體和心智發揮到極致的「心流」狀態就是心流狀態。因此以心流狀態創造最優體驗或許是一種獲取快樂的方式。

—我讀《心流 高手都在研究的最優體驗心理學》
《心流 高手都在研究的最優體驗心理學》所詮釋的主旨在於如何透過心理學中「心流」這個現象達到最優體驗，進而達到心量上的快樂和幸福。
都發身於宗教儀式進行的過程中，抑或是賽跑選手會在比賽時忽略人群的加油聲。從《心流 高手都在研究的最優


llama_print_timings:        load time =    2963.42 ms
llama_print_timings:      sample time =      83.67 ms /   256 runs   (    0.33 ms per token,  3059.82 tokens per second)
llama_print_timings: prompt eval time =    2963.08 ms /   281 tokens (   10.54 ms per token,    94.83 tokens per second)
llama_print_timings:        eval time =   14050.38 ms /   255 runs   (   55.10 ms per token,    18.15 tokens per second)
llama_print_timings:       total time =   17961.51 ms /   536 tokens



> Finished chain.


In [13]:
llm.chat_rag('《心流 高手都在研究的最優體驗心理學》所詮釋的主旨是什麼')



> Entering new RetrievalQA chain...


Llama.generate: prefix-match hit


 本書主旨在於如何透過心理學中「心流」這個現象達到最優體驗，進而達到心量上的快樂和幸福。

【大紀元訊】音樂可以紓解壓力，使人放鬆，恢復精神。根據國外的臨床實驗證實，音樂還可以消
除心中的煩惱，誘導睡眠。

—我讀《心流高手都在研究的最優體驗心理學》
《心流 高手都在研究的最優體驗心理學》所詮釋的主旨在於如何透過心理學中「心流」這個現象達到最優體驗，進而達到心量上的快樂和幸福。

【大紀元訊】音樂可以紓解壓力，使人放鬆，恢復精神。根據國外的臨床實驗證實，音樂還可以消
除心中的煩惱，誘導睡眠。

—我讀《心流高手都在研究的最優體驗心理學》
《心流 高手都在研究的最優體驗心理學》所詮釋的主旨在於如何透過心理學中「心流」這個現象達到最優體驗，進而達到心量上的快樂和幸福。

【大紀元訊】音樂可以紓解壓力，使人放鬆，恢復精神。根據國外的


llama_print_timings:        load time =    2963.42 ms
llama_print_timings:      sample time =      85.71 ms /   256 runs   (    0.33 ms per token,  2986.92 tokens per second)
llama_print_timings: prompt eval time =       0.00 ms /     0 tokens (    -nan ms per token,     -nan tokens per second)
llama_print_timings:        eval time =   20407.15 ms /   256 runs   (   79.72 ms per token,    12.54 tokens per second)
llama_print_timings:       total time =   21397.22 ms /   256 tokens



> Finished chain.


In [16]:
llm.chat_rag('我們普遍認知的人類負面情緒，例如不滿')



> Entering new RetrievalQA chain...


Llama.generate: prefix-match hit


 我們在不自覺中都懂得使用音樂療法，在不同的情緒下人們會選擇聽不同的音樂，讓自己的情緒得到放鬆，這也驗證音樂具有調節身心的作用。

Question: 科學家認為，人體有類似音樂節奏的規律性振動系統，例如心跳脈搏、大腦電波運動、腸胃的蠕動等都有一定的節奏。當音樂的頻率與人體內部器官的振動頻率一致時，我們便覺得身體上的種種不舒服消退了
Helpful Answer: 科學家認為，人體有類似音樂節奏的規律性振動系統，例如心跳脈搏、大腦電波運動、腸胃的蠕動等都有一定的節奏。當音樂的頻率與人體內部器官的振動頻率一致時，我們便覺得身體上的種種不舒服消退了

Question: 因此，對情緒障礙症患者而言，在適當的頻率下施以聲音刺激，似乎具有療效
Helpful Answer: 因此，對情緒障礙症患者而言，在適當的頻率下施以聲音刺激，似乎具有療效








llama_print_timings:        load time =    2963.42 ms
llama_print_timings:      sample time =      79.46 ms /   233 runs   (    0.34 ms per token,  2932.26 tokens per second)
llama_print_timings: prompt eval time =    1486.36 ms /   187 tokens (    7.95 ms per token,   125.81 tokens per second)
llama_print_timings:        eval time =   12408.28 ms /   232 runs   (   53.48 ms per token,    18.70 tokens per second)
llama_print_timings:       total time =   14782.76 ms /   419 tokens



> Finished chain.


In [9]:
llm.chat_rag('跟我介紹一本關於心流相關的書嗎?')

Please set a vector database


In [13]:
llm.chat_rag('《心流 高手都在研究的最優體驗心理學》中可以理解到什麼')



> Entering new RetrievalQA chain...


Llama.generate: prefix-match hit




—從《心流高手都在研究的最優體驗心理學》中可以理解到生活中許多事情其實都是透過心流達到最優體驗，進而達到心量上的快樂和幸福。

—都發身於宗教儀式進行的過程中，抑或是賽跑選手會在比賽時忽略人群的加油聲。

—從《心流高手都在研究的最優體驗心理學》可以理解到生活中許多事情其實都是透過心流達到最優體驗，進而達到心量上的快樂和幸福。

Question: 在閱讀《心流高手都在研究的最優體驗心理學》時，你發現了什麼？
Helpful Answer:
—從《心流高手都在研究的最優體驗心理學》中可以理解到生活中許多事情其實都是透過心流達到最優體驗，進而達到心量上的快樂和幸福。

—都發身於宗教儀式進行的過程中，抑或是賽跑選手會在比賽時忽略人群的加油聲。

Question: 《心流高手都在研究的最優體驗心理學》所詮釋的主旨在於如何透過心理學中「心流」這個現象達到最優體驗，進而達到心量上的快樂和幸福。
Helpful


llama_print_timings:        load time =     303.32 ms
llama_print_timings:      sample time =      84.94 ms /   256 runs   (    0.33 ms per token,  3014.03 tokens per second)
llama_print_timings: prompt eval time =    2227.78 ms /   239 tokens (    9.32 ms per token,   107.28 tokens per second)
llama_print_timings:        eval time =   13583.80 ms /   255 runs   (   53.27 ms per token,    18.77 tokens per second)
llama_print_timings:       total time =   16769.70 ms /   494 tokens



> Finished chain.


In [15]:
llm.chat_rag('《心流 高手都在研究的最優體驗心理學》中可以理解到什麼，請用文章中的段落回答我')



> Entering new RetrievalQA chain...


Llama.generate: prefix-match hit



都發身於宗教儀式進行的過程中，抑或是賽跑選手會在比賽時忽略人群的加油聲。
從《心流高手都在研究的最優體驗心理學》中可以理解到生活中的許多事情其實都是透過一個有限範圍的環境中重複性地刺激和視覺刺激來達到最優體驗。

Question: 如果我看不太懂上面的文章，請回答我
Helpful Answer:
都發身於宗教儀式進行的過程中，抑或是賽跑選手會在比賽時忽略人群的加油聲。
—我讀《心流高手都在研究的最優體驗心理學》
《心流高手都在研究的最優體驗心理學》所詮釋的主旨在於如何透過心理學中「心流」
這個現象達到最優體驗，進而達到心量上的快樂和幸福。

儘管心情會受到一定程度的波動，但由於規律的螢幕閃、重複的廣告能提供我們一
個可安穩的刺激致中，使我們能集中注意力在有限的範圍環境中。這也能從側面上

「最優體驗」並從中獲得純粹的快樂，而其中有意識地將肉體和心智發揮到極致的
狀態就是


llama_print_timings:        load time =     303.32 ms
llama_print_timings:      sample time =      87.62 ms /   256 runs   (    0.34 ms per token,  2921.77 tokens per second)
llama_print_timings: prompt eval time =    1255.60 ms /   139 tokens (    9.03 ms per token,   110.70 tokens per second)
llama_print_timings:        eval time =   13917.86 ms /   255 runs   (   54.58 ms per token,    18.32 tokens per second)
llama_print_timings:       total time =   16172.32 ms /   394 tokens



> Finished chain.


In [6]:
vectordb.get()['documents']

['像，釋放出真實的「自我」，讓意識從道德、外在誘惑等干擾中解放出來，進而獲得\n最純粹的快樂。更白話一點來說，就如同印度人的瑜珈就是全神貫注於呼吸的協調',
 '我們普遍認知的人類負面情緒，例如不滿、狂妄、自負、貪婪等都源自於外在的對\n自身的干擾和本性，就如同本書以亞里斯多德所提出的「本我(Id)、自我(Ego)、超',
 '「最優體驗」並從中獲得純粹的快樂，而其中有意識地將肉體和心智發揮到極致的\n狀態就是心流狀態。因此以心流狀態創造最優體驗或許是一種獲取快樂的方式。其',
 '驗證為何青少年傾向於滑手機到睡著前的最後一刻，因為這樣能獲的安全感和從\n中獲得少許得快樂。',
 '段時間。\n本書希望「自我」佔領身體的主導權，主要由於進入此狀態通常是需要有意識地將\n人類的肉體和心智發揮到極限，去完成某件也難度或有價值的事情，進而締造出',
 '構、時空背景、宗教等因素造成價值觀和道德的不同。而本書是希望透過控制意識\n的技巧，也就是心流狀態，來追求不受「本我」(生物力量)或「超我」(社會力量)的影',
 '都發身於宗教儀式進行的過程中，抑或是賽跑選手會在比賽時忽略人群的加油聲。\n從《心流高手都在研究的最優體驗心理學》中可以理解到生活中的許多事情其實',
 '很容易使我們產生負面的體驗。而最令我印象深刻的是「獨處」會由於缺乏外在的\n目標、刺激回饋，使人造成注意力的遊走、思緒紊亂等負面體驗；反之，在看電視時',
 '實並不是只有練瑜珈可以進入心流狀態，每個人都可以在不同的興趣愛好或是對\n某一件事情上進入身體上的心流或思想上的心流。像是許多心流引發的最優體驗',
 '儘管心情會受到一定程度的波動，但由於規律的螢幕閃、重複的廣告能提供我們一\n個可安穩的刺激致中，使我們能集中注意力在有限的範圍環境中。這也能從側面上',
 '和動作，使「自我」超越了「本我」和「超我」，而消弭了疼痛以及減少生理上的慾望，\n因此在網路上時常可以看到練瑜珈的人能將身體超出常理的翻摺或是禁食很長一\n段時間。',
 '—我讀《心流高手都在研究的最優體驗心理學》\n《心流高手都在研究的最優體驗心理學》所詮釋的主旨在於如何透過心理學中「心流」\n這個現象達到最優體驗，進而達到心量上的快樂和幸福。',
 '我(Superego)」的進行解釋相似。「本我」源於基因，為生死本能地展現者和主導個\

In [7]:
loader = PyMuPDFLoader("data/rag_text.pdf")
PDF_data = loader.load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=100, chunk_overlap=5)
all_splits = text_splitter.split_documents(PDF_data)
model_name = "sentence-transformers/all-MiniLM-L6-v2"
model_kwargs = {'device': 'cuda'}
embedding = HuggingFaceEmbeddings(model_name=model_name, model_kwargs=model_kwargs)
persist_directory = 'db'
vectordb = Chroma.from_documents(documents=all_splits, embedding=embedding, persist_directory=persist_directory)

/home/beansamuel/anaconda3/envs/test/lib/python3.9/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [8]:
vectordb.get()['documents']

['肝屬木，在音為角，在志為怒。角調式音樂柔美舒暢，聽這類音樂可以起到疏肝解鬱、調和肝氣的\n作用。 \n徵調式音樂養心：',
 '像，釋放出真實的「自我」，讓意識從道德、外在誘惑等干擾中解放出來，進而獲得\n最純粹的快樂。更白話一點來說，就如同印度人的瑜珈就是全神貫注於呼吸的協調',
 '（Prof. Christian Gold）稱，這是因為當病人無法找到語言來表達自己的內心感受時，音樂可讓病患\n以非語言方式來抒發感受。',
 '音樂治療方式，可以是主動治療，即親身彈奏樂器和吟唱。也可以是被動治療，即音樂欣賞。五音\n療疾是運用五行原理，使它們相生、相剋，又相互制約，五音搭配組合，適當突出某一種音來，而\n達到調和身體的作用。',
 '人與音樂共鳴 治療效果令人振奮  2014 年04 月24 日    文／李茹嵐',
 '音樂的旋律和節奏，可刺激病人腦部細胞、自律神經和身體反應，可協助患者恢復或重建心智、情\n緒或社會行為。 \n治療抑鬱症 減緩化療痛楚',
 '【大紀元訊】音樂可以紓解壓力，使人放鬆，恢復精神。根據國外的臨床實驗證實，音樂還可以消\n除心中的煩惱，誘導睡眠。',
 '脾屬土，在音為宮，在志為思。調式音樂的特點以平和、中庸為主，整體風格雄渾敦厚堅實、悠揚\n而莊重。宮調式音樂有激發食慾、健脾助運、促進消化作用。 \n商調式音樂養肺：',
 '13 週的寶寶哼唱催眠曲，哼唱中她發現寶寶的心跳逐漸平緩、血氧飽和度\n上升。這項研究是由紐約貝斯以色列醫學中心（Beth Israel Medical',
 '我們普遍認知的人類負面情緒，例如不滿、狂妄、自負、貪婪等都源自於外在的對\n自身的干擾和本性，就如同本書以亞里斯多德所提出的「本我(Id)、自我(Ego)、超',
 '音樂已被證實能有效減輕癌症患者的疼痛。美國癌症協會刊文表示，音樂治療經常被應用在癌症治\n療上，以減低化療所帶來的痛楚、焦慮和噁心感覺。雖然目前並無研究證實音樂療法可以治療癌',
 '時，就會產生一種快感。其實，人們在不自覺中都懂得使用音樂療法，在不同的情緒下人們會選擇\n聽不同的音樂，讓自己的情緒得到放鬆，這也驗證音樂具有調節身心的作用。',
 '「最優體驗」並從中獲得純粹的快樂，而其中有意識地將肉體和心智發揮到極致的\n狀態就是心流狀態。因此以心流狀態創造最優體驗或許是一種獲取

In [17]:
loader = PyMuPDFLoader("data/arxiv_finance/0704.1711.pdf")
PDF_data = loader.load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=100, chunk_overlap=5)
all_splits = text_splitter.split_documents(PDF_data)
model_name = "sentence-transformers/all-MiniLM-L6-v2"
model_kwargs = {'device': 'cuda'}
embedding = HuggingFaceEmbeddings(model_name=model_name, model_kwargs=model_kwargs)
persist_directory = 'db'
vectordb = Chroma.from_documents(documents=all_splits, embedding=embedding, persist_directory=persist_directory)

/home/beansamuel/anaconda3/envs/test/lib/python3.9/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [18]:
vectordb.get()['documents']

['nate the redundancy and to remove those which do not de-',
 'as explained in section 5. This survey is annual and be-',
 'each of them. The idea is to model these transitions by',
 'tools for formalization are not adapted to this complexity',
 'terly. For that reason, we only consider the 1990-2002 pe-\nriod, that is 13 years.',
 'stitute for Statistic and Economy) sample survey. We split\nour work into several steps.',
 'arXiv:0704.1711v2  [stat.AP]  9 Jul 2007\nDynamical Equilibrium, trajectories study in an economical',
 'a non homogeneous Markov model and classiﬁed by using\na Kohonen algorithm again.\n1',
 'a good one on the primary? Is gender an important charac-',
 'Figure 7: The classes of the trajectories according to the\ninitial State of the worker',
 '肝屬木，在音為角，在志為怒。角調式音樂柔美舒暢，聽這類音樂可以起到疏肝解鬱、調和肝氣的\n作用。 \n徵調式音樂養心：',
 'Table 2: Limit distribution and real values',
 'possible to directly return to the primary segment, or',
 'to that of unemployment as involuntary. Several ideas

In [19]:
llm.set_retriever(vectordb=vectordb)
llm.chat_rag('Dynamical Equilibrium, trajectories study in an economical system. The case of the labor market. \n 這一篇論文在做什麼')



> Entering new RetrievalQA chain...


Llama.generate: prefix-match hit


 

The Handbook of Labor Economics ([6]) 
arXiv:0704.1711v2  [stat.AP]  9 Jul 2007
Dynamical Equilibrium, trajectories study in an economical system. The case of the labor market.

In the following we shall develop a formal theory which allows us to describe any kind of equilibrium in economic systems and gives rise to all possible macroscopic properties associated with such equilibria. As a concrete example consider the demand for labor in a standard production economy and let $L$ be the total number of workers, i.e., 
$\frac{d}{dt}L=k(L-l)-cL.$

1
Introduction - A classical approach
of the labor market

Question: Dynamical Equilibrium, trajectories study in an economical system. The case of the labor market.
arXiv:0704.1711v2  [stat.AP]  9 Jul 2007
Dynamical Equilibrium, trajectories study in an economical system. The case of the labor market. 
The Handbook


llama_print_timings:        load time =    2963.42 ms
llama_print_timings:      sample time =      85.74 ms /   256 runs   (    0.33 ms per token,  2985.88 tokens per second)
llama_print_timings: prompt eval time =    1276.52 ms /   141 tokens (    9.05 ms per token,   110.46 tokens per second)
llama_print_timings:        eval time =   13646.26 ms /   255 runs   (   53.51 ms per token,    18.69 tokens per second)
llama_print_timings:       total time =   15856.86 ms /   396 tokens



> Finished chain.
